In [ ]:
pip install opencv-python mtcnn tensorflow torch

In [ ]:
import cv2
from mtcnn.mtcnn import MTCNN
import numpy as np
import torch
from torchvision import transforms
from arcface.model import ArcFaceModel

In [ ]:
def load_mtcnn_model():
    return MTCNN()

mtcnn_model = load_mtcnn_model()


In [ ]:
def load_arcface_model():
    model = ArcFaceModel(size=112, backbone_type='ResNet50', model_path='path/to/arcface_model_weights.pth')
    return model

arcface_model = load_arcface_model()
arcface_model.eval()


In [ ]:
def preprocess_image(img):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    return transform(img)

def face_recognition(frame):
    # Detect faces using MTCNN
    faces = mtcnn_model.detect_faces(frame)
    
    for face in faces:
        x, y, w, h = face['box']
        roi = frame[y:y + h, x:x + w]

        # Preprocess the face for ArcFace model
        input_face = preprocess_image(roi)
        input_face = input_face.unsqueeze(0)

        # Extract features using ArcFace
        with torch.no_grad():
            embedding = arcface_model(input_face)

        # Perform face recognition using the embeddings (you'll need a database for this)
        recognized_person = recognize_person(embedding)

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, recognized_person, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return frame

def recognize_person(embedding):
    # Implement your logic for face recognition based on the embeddings
    # You might compare the embeddings with a database of known faces
    # and return the name or ID of the recognized person.
    return "John Doe"

# Open a video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform face recognition on the frame
    frame = face_recognition(frame)

    # Display the result
    cv2.imshow('Face Recognition', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import os
import joblib


In [ ]:
def load_or_train_svm_model():
    svm_model_path = 'path/to/your/svm_model.pkl'

    if os.path.exists(svm_model_path):
        # Load pre-trained SVM model
        svm_model = joblib.load(svm_model_path)
        label_encoder = joblib.load('path/to/your/label_encoder.pkl')
    else:
        # Train SVM model using your dataset
        # Assume you have a list of face embeddings (X) and corresponding labels (y)
        # X and y should be obtained from your face recognition database
        # You need to collect embeddings and labels for known individuals
        X, y = collect_embeddings_and_labels()

        # Encode labels
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)

        # Train SVM model
        svm_model = SVC(C=1.0, kernel='linear', probability=True)
        svm_model.fit(X, y_encoded)

        # Save the trained model and label encoder for future use
        joblib.dump(svm_model, svm_model_path)
        joblib.dump(label_encoder, 'path/to/your/label_encoder.pkl')

    return svm_model, label_encoder


In [ ]:
def collect_embeddings_and_labels():
    # Replace this with your logic to collect face embeddings and labels
    # You need to have a list of face embeddings (X) and corresponding labels (y)
    # X and y should be obtained from your face recognition database
    # Each row of X should be an embedding, and each element of y should be the label for that embedding
    X = []
    y = []

    # Example: Iterate through your dataset and append embeddings and labels
    # for img_path, label in your_dataset:
    #     embedding = compute_embedding(img_path)  # You need to implement this
    #     X.append(embedding)
    #     y.append(label)

    return np.array(X), np.array(y)


In [ ]:
def recognize_person(embedding, svm_model, label_encoder):
    # Use the SVM model to predict the label for the given embedding
    probas = svm_model.predict_proba([embedding])
    predicted_class = np.argmax(probas)
    confidence = probas[0, predicted_class]

    # Convert the predicted class back to the original label using the label encoder
    recognized_person = label_encoder.inverse_transform([predicted_class])[0]

    # You can use confidence to set a threshold for recognition confidence
    # For example, only recognize if confidence is above a certain threshold
    threshold = 0.8
    if confidence < threshold:
        recognized_person = "Unknown"

    return recognized_person


In [ ]:
# ...

# Load or train SVM model
svm_model, label_encoder = load_or_train_svm_model()

# ...

def face_recognition(frame):
    # ...

    for face in faces:
        # ...

        # Perform face recognition using the SVM model
        recognized_person = recognize_person(embedding, svm_model, label_encoder)

        # ...
